In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("data/my.csv", index_col=None,header=None)#do for test too

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [8]:
y = df[df.columns[-1]].values
from collections import Counter
#num = Counter(y)
#n=num.get('normal')
Counter(y)

Counter({'normal': 67343,
         'neptune': 41214,
         'warezclient': 890,
         'ipsweep': 3599,
         'portsweep': 2931,
         'teardrop': 892,
         'nmap': 1493,
         'satan': 3633,
         'smurf': 2646,
         'pod': 201,
         'back': 956,
         'guess_passwd': 53,
         'ftp_write': 8,
         'multihop': 7,
         'rootkit': 10,
         'buffer_overflow': 30,
         'imap': 11,
         'warezmaster': 20,
         'phf': 4,
         'land': 18,
         'loadmodule': 9,
         'spy': 2,
         'perl': 3})

In [9]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in df.columns:
    #print df[c].dtype
    if df[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        df[c] = encodings[c].fit_transform(df[c])

In [11]:
y = df.pop(df.columns[-1]).values
X = df.values

In [15]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer([('t', OneHotEncoder(), [1])], remainder='passthrough')
X = onehotencoder.fit_transform(X)


In [16]:
from sklearn.preprocessing import StandardScaler #normalization
X = StandardScaler().fit_transform(X)

In [17]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDfX = pd.DataFrame(data = principalComponents)

In [18]:
principalDfX.head()
print(Counter(y))

Counter({11: 67343, 9: 41214, 17: 3633, 5: 3599, 15: 2931, 18: 2646, 10: 1493, 0: 956, 20: 892, 21: 890, 14: 201, 3: 53, 1: 30, 22: 20, 6: 18, 4: 11, 16: 10, 7: 9, 2: 8, 8: 7, 13: 4, 12: 3, 19: 2})


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(principalDfX, y, test_size=0.2, random_state=42)#random for splitting same data when run again.
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [20]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(random_state=0)

In [22]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(base_estimator=clf1,n_estimators=10,random_state=0)
clf.fit(X_train, y_train) 

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=0),
                   n_estimators=10, random_state=0)

In [23]:
y_pred = clf.predict(X_test) # test the output 
print(Counter(y_pred))
print(Counter(y_test))

Counter({11: 13456, 9: 8225, 5: 736, 17: 725, 15: 575, 18: 530, 10: 319, 0: 194, 20: 182, 21: 181, 14: 40, 3: 10, 22: 7, 4: 4, 6: 3, 1: 3, 2: 2, 7: 1, 8: 1, 16: 1})
Counter({11: 13422, 9: 8228, 17: 738, 5: 733, 15: 573, 18: 534, 10: 313, 21: 202, 20: 188, 0: 185, 14: 43, 3: 11, 1: 9, 22: 8, 6: 3, 4: 1, 19: 1, 16: 1, 13: 1, 12: 1})


In [24]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, y_pred))
print('Report : ')
print(classification_report(y_test, y_pred))


Confusion Matrix :
[[  110     0     1     0     0     0     0     0     0     0     0    64
      0     0     0     0     0     0     6     0     2     2     0]
 [    0     0     0     0     0     0     0     0     0     0     0     8
      0     0     0     0     0     0     0     0     0     1     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     8     0     0     0     0     0     0     0     2
      0     0     0     1     0     0     0     0     0     0     0]
 [    0     0     0     0     1     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0   696     0     0     0     0    14    17
      0     0     1     5     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     1     0     2
      0     0     0     0     0     0     0     0

D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
predicted = cross_val_predict(clf, X_train, y_train, cv=skf)
print('Accuracy Score :',accuracy_score(y_train, predicted))
print('Report : ')
print(classification_report(y_train, predicted))


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy Score : 0.9564785965190815
Report : 
              precision    recall  f1-score   support

           0       0.59      0.56      0.57       771
           1       0.24      0.19      0.21        21
           2       0.40      0.25      0.31         8
           3       0.78      0.83      0.80        42
           4       0.00      0.00      0.00        10
           5       0.95      0.96      0.96      2866
           6       0.12      0.07      0.09        15
           7       0.00      0.00      0.00         9
           8       0.17      0.14      0.15         7
           9       1.00      1.00      1.00     32986
          10       0.91      0.91      0.91      1180
          11       0.96      0.97      0.96     53921
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         3
          14       0.69      0.68      0.68       158
          15       0.94      0.93      0.93      2358
          16       0.00      0.00  